In [9]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
from model import CropYieldModel, CropDataset


## 데이터 전처리 및 로딩

In [10]:
# 데이터 전처리 및 로딩
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = CropDataset('data/json_directory/', 'data/images_directory/', transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


Creating dataset from data/json_directory/ and data/images_directory/
Found 50 json files
Loaded 50 valid samples out of 50 JSON files


## 모델, 손실 함수, 옵티마이저 초기화 

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CropYieldModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 학습루프

In [12]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for i, (images, features, labels) in enumerate(dataloader):
        images = images.to(device)
        features = features.to(device)
        labels = labels.to(device)

        outputs = model(images, features)
        loss = criterion(outputs, labels.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 134096.7969
Epoch [2/100], Loss: 148828.5312
Epoch [3/100], Loss: 128878.5938
Epoch [4/100], Loss: 78818.5312
Epoch [5/100], Loss: 40438.9922
Epoch [6/100], Loss: 43283.9102
Epoch [7/100], Loss: 44510.6719
Epoch [8/100], Loss: 30282.8711
Epoch [9/100], Loss: 38721.8047
Epoch [10/100], Loss: 21593.6465
Epoch [11/100], Loss: 16496.5039
Epoch [12/100], Loss: 14210.1367
Epoch [13/100], Loss: 9638.5029
Epoch [14/100], Loss: 18299.3477
Epoch [15/100], Loss: 19735.9414
Epoch [16/100], Loss: 20505.0918
Epoch [17/100], Loss: 11380.7598
Epoch [18/100], Loss: 18356.6328
Epoch [19/100], Loss: 15967.2207
Epoch [20/100], Loss: 8315.0557
Epoch [21/100], Loss: 11021.5527
Epoch [22/100], Loss: 4402.8779
Epoch [23/100], Loss: 7424.5435
Epoch [24/100], Loss: 9836.5332
Epoch [25/100], Loss: 15116.0850
Epoch [26/100], Loss: 9320.3086
Epoch [27/100], Loss: 12810.1807
Epoch [28/100], Loss: 8308.9814
Epoch [29/100], Loss: 6378.6738
Epoch [30/100], Loss: 12898.9023
Epoch [31/100], Loss: 32

In [13]:
torch.save(model.state_dict(), 'model/crop_yield_model.pth')